# Steps forward:

* SEND UPDATE ABOUT NYC WHEN TEST CASE IS DONE

## effecient node edge creation
* ~~Make sure the functionality to construct nodes-edges works for a safey buffer~~
* ~~Isolate the node insertion code into script~~
* ~~commit script to my branch~~
* ~~email orion about it, mention how easy to integrate.~~
* look into issue with redundant edges

## Testing
* ~~structure tests so its a one csv: first n rows are settings, next m rows are edges flows, with first column being same segment ids from network file~~
* ~~standard test case is four files: network.jeojson - origins.geojson - destination.jeojson - testflows.csv~~
* ~~function reads a csv into list of dicts: test settings and series of output.~~
* ~~build Harvard Square testflows.csv~~
~~* make sure this config runs on Harvard Square~~
* build Manhattan testflow.csv

## Documentation
* set up a slideshow/diagram to show the relationship between different compoonents of the library.
* Set up a notebook to go over estimating flows from one origin to one destination.
* Document all relevant settings
* show how this generalizes to the pairings.csv
* show how this generalizes to iterating over moodel settings
* show how this generalizes to iterating over scenarios

## input structure
maybe some parameters are global to zonal. Updated only when needed, instead of passed as inputs?

## output structure
* include Logger as a class, and zonal would have an instance of that class. Logger handles event documentation, and captures output.
* think clearly about the above usecases and structure output accordingly
* for each origin, have an origin record, showing its knn weight, reach, gravity towards each of its destination, header rows showing settings and parameters
* for the network, columns of od flows, headed  by settings and parameters rows that explicitly detail units, weight type, calibration status, .. any relevant data, settings and parameters..



In [1]:
import os
import geopandas as gpd
import pandas as pd
import sys
import math


sys.path.append('../')
from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight

for test_case in os.listdir("Test Cases"):
    # TODO: Check OS compatibility, ensure this is compatible with Unix systems..
    test_case_folder = "Test Cases" + "\\" + test_case + "\\"
    test_config = pd.read_csv(test_case_folder + "test_configs.csv")
    test_flows =  pd.read_csv(test_case_folder + "test_flows.csv")

    harvard_square = Zonal(projected_crs='EPSG:3857')

    harvard_square.load_layer(
        layer_name='streets',
        file_path=  test_case_folder + test_config.at[0, 'Network_File']
        )

    harvard_square.load_layer(
        layer_name=test_config.at[0, 'Origin_Name'],
        file_path= test_case_folder + test_config.at[0, 'Origin_File']
        )

    harvard_square.load_layer(
        layer_name=test_config.at[0, 'Destination_Name'],
        file_path= test_case_folder + test_config.at[0, 'Destination_File']
        )
    
    harvard_square.create_street_network(
        source_layer='streets', 
        discard_redundant_edges=False
    )

    harvard_square.insert_node(
        layer_name=test_config.at[0, 'Origin_Name'], 
        label='origin', 
        weight_attribute=test_config.at[3, 'Origin_Weight']
    )

    harvard_square.insert_node(
        layer_name=test_config.at[0, 'Destination_Name'], 
        label='destination', 
        weight_attribute=test_config.at[3, 'Destination_Weight']
    )

    harvard_square.create_graph(light_graph=True, d_graph=True)

    node_gdf = harvard_square.network.nodes
    origin_gdf = node_gdf[node_gdf['type'] == 'origin']

    harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]


    # ["original_weight", "elastic_weight", "knn_weight"]

    for test_idx in test_config.index:
        #harvard_square.network.turn_penalty_amount = 0
        #harvard_square.network.turn_threshold_degree = 0

        if test_config.at[test_idx, 'Elastic_weights']:
            harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
            get_elastic_weight(
                harvard_square.network,
                search_radius=test_config.at[test_idx, 'Radius'],
                detour_ratio=test_config.at[test_idx, 'Detour'],
                beta=test_config.at[test_idx, ' Beta '],
                decay=True, #test_config.at[test_idx, 'Decay'],
                #turn_penalty=test_config.at[test_idx, 'Turns'],
                turn_penalty=False,
            )
            for o_idx in origin_gdf.index:
                harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']


        return_dict = parallel_betweenness(
            harvard_square.network,
            search_radius=test_config.at[test_idx, 'Radius'],
            detour_ratio=test_config.at[test_idx, 'Detour'],
            decay=test_config.at[test_idx, 'Decay'], #if test['Elastic weights'] else True,
            decay_method=test_config.at[test_idx, 'Decay_Mode'],  # "power", "exponent"
            beta=test_config.at[test_idx, ' Beta '],
            path_detour_penalty="equal",  # "power", "exponent", "equal"
            origin_weights=False if type(test_config.at[test_idx, 'Origin_Weight']) != str else True,
            closest_destination=test_config.at[test_idx, 'Closest_destination'],
            destination_weights=False if type(test_config.at[test_idx, 'Destination_Weight']) != str  else True,    #or (test['Elastic weights'])
            # perceived_distance=False,
            num_cores=2,
            light_graph=True,
            turn_penalty=test_config.at[test_idx, 'Turns'],
        )
        simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()
        test_flow = test_flows[test_config.at[test_idx, 'test_name']].sum()

        print (test_config.loc[test_idx])
        print (f"{test_config.at[test_idx, 'test_name']}\t\t{simulated_sum_of_flow = }\t test flow = { test_flow }\t difference = {simulated_sum_of_flow - test_flow}\t similarity {1-(simulated_sum_of_flow - test_flow)/ test_flow:.2%}")

c:\Users\abdul\.conda\envs\madina_dev\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\abdul\AppData\Local\Temp\ipykernel_3540\1977593790.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readth


Position 1 ----- streets

Position 2 ----- Buildings
Position 1 ----- streets

Position 3 ----- Subway
Position 2 ----- Buildings
Position 1 ----- streets


KeyError: 'Key network_nodes does not exist in Zonal. Remember: only existing layers can be re-set by label'

In [6]:
harvard_square.layers['streets'].gdf

,__Length,__GUID,geometry,id
0,53.328770,1faf3b03-2e30-44b2-8b28-f84da30193c4,"LINESTRING (-1719.114 147.249, -1718.693 124.0...",0
1,33.137771,1956c1b1-6c7b-46c6-be18-630210c0c086,"LINESTRING (-1705.465 177.057, -1714.466 163.1...",1
2,82.471466,7a8f2a5b-e209-4b06-9c03-19df15c2e86c,"LINESTRING (-1635.552 240.490, -1555.262 259.331)",2
3,20.707448,65e6f380-1774-4439-9478-d23c97aa8346,"LINESTRING (-1648.164 226.821, -1647.328 234.0...",3
4,60.851523,a77163e9-5762-457c-8dda-99b4cfb29da4,"LINESTRING (-1662.239 245.651, -1696.978 295.612)",4
...,...,...,...,...
165,15.383524,db2d5c42-357e-4ef4-ac0d-c01414466159,"LINESTRING (-1651.361 256.529, -1662.239 245.651)",165
166,22.520863,b561650f-19dc-4496-a312-b9ecfebf9fbd,"LINESTRING (-1651.361 256.529, -1635.552 240.490)",166
167,21.412938,53692ac7-ccd5-4135-a75b-58540acdb01a,"LINESTRING (-1680.556 208.784, -1691.978 226.897)",167
168,23.915390,0097e461-14b4-4198-9d46-91766699850b,"LINESTRING (-1741.957 154.332, -1719.114 147.249)",168


In [2]:
import geopandas as gpd

files = [
    r"C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\Home_PT_6538.geojson", 
    r"C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\Metro_PT_6538.geojson", 
    r"C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\network_clipped_dupremovedAS.geojson"
    ]


for file in files:
    gdf = gpd.read_file(file, engine='pyogrio')
    print (file)
    print (gdf.crs)

c:\Users\abdul\.conda\envs\madina_dev\Lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\abdul\AppData\Local\Temp\ipykernel_21008\756385262.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readth

C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\Home_PT_6538.geojson
EPSG:6538
C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\Metro_PT_6538.geojson
EPSG:6538
C:\Users\abdul\Dropbox (MIT)\115_NYCWalks\03_Data\03_Model\Cities\NYC_val500\Data\network_clipped_dupremovedAS.geojson
EPSG:6538


In [3]:
buildings_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Home_PT_6538.geojson"
subway_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\Metro_PT_6538.geojson"
network_file = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\madina\unit_testing\Test Cases\Manhattan\network_clipped_dupremovedAS.geojson"
\
import sys
sys.path.append('../')
from madina.zonal.zonal import Zonal
harvard_square = Zonal(projected_crs='EPSG:6538')

harvard_square.load_layer(
    layer_name='streets',
    file_path=network_file
    )


harvard_square.load_layer(
    layer_name='buildings',
    file_path=buildings_file
    )


harvard_square.load_layer(
    layer_name='subway',
    file_path=subway_file
    )



harvard_square.create_street_network(
    source_layer='streets', 
    discard_redundant_edges=False
)

harvard_square.insert_node(
    layer_name='buildings', 
    label='origin', 
    weight_attribute='TotalPop'
)

harvard_square.insert_node(
    layer_name='subway', 
    label='destination', 
    weight_attribute='line_ent_st'
)

harvard_square.create_graph(light_graph=True, d_graph=True)

node_gdf = harvard_square.network.nodes
origin_gdf = node_gdf[node_gdf['type'] == 'origin']

harvard_square.network.nodes["original_weight"] = harvard_square.network.nodes["weight"]
# ["original_weight", "elastic_weight", "knn_weight"]




Position 1 ----- streets

Position 2 ----- buildings
Position 1 ----- streets

Position 3 ----- subway
Position 2 ----- buildings
Position 1 ----- streets
counter = 100, progress =  0.18
counter = 200, progress =  0.36
counter = 300, progress =  0.54
counter = 400, progress =  0.72
counter = 500, progress =  0.90
counter = 600, progress =  1.08
counter = 700, progress =  1.26
counter = 800, progress =  1.44
counter = 900, progress =  1.62
counter = 1000, progress =  1.80
counter = 1100, progress =  1.98
counter = 1200, progress =  2.16
counter = 1300, progress =  2.34
counter = 1400, progress =  2.52
counter = 1500, progress =  2.70
counter = 1600, progress =  2.88
counter = 1700, progress =  3.06
counter = 1800, progress =  3.24
counter = 1900, progress =  3.42
counter = 2000, progress =  3.60
counter = 2100, progress =  3.78
counter = 2200, progress =  3.96
counter = 2300, progress =  4.14
counter = 2400, progress =  4.32
counter = 2500, progress =  4.51
counter = 2600, progress =  

KeyboardInterrupt: 

In [ ]:
tests = [
    {
        'test_name': '_Betweenness1',
        'Origin weight': None, 
        'Destination weight': None,
        'Search radius': 800,
        'beta': 0.004,
        'Detour Ratio': 1.15,
        'Elastic weights': False, 
        'Turns': False,
        'Turn threshold': 0, 
        'Turn penalty':	0,
        'sum of flow': 2247.861132
    }, 
    '''
    {
        'test_name': '_Betweenness2',
        'Origin weight': 'people', 
        'Destination weight': None,
        'Search radius': 300,
        'beta': 0.004,
        'Detour Ratio': 1,
        'Elastic weights': False, 
        'Turns': False,
        'Turn threshold': 0, 
        'Turn penalty':	0,
        'sum of flow': 19025.62776
    }, 
    {
        'test_name': '_Betweenness3',
        'Origin weight': 'people', 
        'Destination weight': 'lines',
        'Search radius': 300,
        'beta': 0.004,
        'Detour Ratio': 1,
        'Elastic weights': False, 
        'Turns': False,
        'Turn threshold': 0, 
        'Turn penalty':	0,
        'sum of flow': 19214
    }, 
    {
        'test_name': '_Betweenness4',
        'Origin weight': 'people', 
        'Destination weight': 'lines',
        'Search radius': 300,
        'beta': 0.004,
        'Detour Ratio': 1.15,
        'Elastic weights': False, 
        'Turns': False,
        'Turn threshold': 0, 
        'Turn penalty':	0,
        'sum of flow': 20853.17809
    }, 
    {
        'test_name': '_Betweenness5',
        'Origin weight': 'people', 
        'Destination weight': 'lines',
        'Search radius': 300,
        'beta': 0.004,
        'Detour Ratio': 1.15,
        'Elastic weights': True, 
        'Turns': False,
        'Turn threshold': 0, 
        'Turn penalty':	0,
        'sum of flow': 13394.55232
    }, 
    {
        'test_name': '_Betweenness6',
        'Origin weight': 'people', 
        'Destination weight': 'lines',
        'Search radius': 300,
        'beta': 0.004,
        'Detour Ratio': 1.15,
        'Elastic weights': True, 
        'Turns': True,
        'Turn threshold': 45, 
        'Turn penalty':	30,
        'sum of flow': 7258.572304
    }
    '''
]



from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight
for test in tests:
    if test['Elastic weights']:
        harvard_square.network.nodes["weight"] = harvard_square.network.nodes["original_weight"]
        get_elastic_weight(
            harvard_square.network,
            search_radius=test['Search radius'],
            detour_ratio=test['Detour Ratio'],
            beta=test['beta'],
            decay=True,
            #turn_penalty=test['Turns'],
            turn_penalty=False,
        )
        for o_idx in origin_gdf.index:
            harvard_square.network.nodes.at[o_idx, 'weight'] =  harvard_square.network.nodes.at[o_idx, 'elastic_weight']


    #harvard_square.network.turn_penalty_amount = 0
    #harvard_square.network.turn_threshold_degree = 0

    return_dict =parallel_betweenness(
        harvard_square.network,
        search_radius=test['Search radius'],
        detour_ratio=test['Detour Ratio'],
        decay=False, #if test['Elastic weights'] else True,
        decay_method="exponent",  # "power", "exponent"
        beta=test['beta'],
        path_detour_penalty="equal",  # "power", "exponent", "equal"
        origin_weights=False if test['Origin weight'] is None else True,
        closest_destination=False,
        destination_weights=False if (test['Destination weight'] is None)  else True,    #or (test['Elastic weights'])
        # perceived_distance=False,
        num_cores=2,
        light_graph=True,
        turn_penalty=test['Turns'],
    )
    simulated_sum_of_flow = return_dict['edge_gdf']['betweenness'].sum()

    print (test)
    print (f"{test['test_name']}\t\t{simulated_sum_of_flow = }\t{test['sum of flow'] = }\t difference = {simulated_sum_of_flow - test['sum of flow']}\t similarity {1-(simulated_sum_of_flow - test['sum of flow'])/ test['sum of flow']:.2%}")

In [ ]:
node_gdf = harvard_square.network.nodes
origin_gdf = node_gdf[node_gdf['type'] == 'origin']
origin_gdf

In [ ]:
import os
import geopandas as gpd
folder = r"C:\Users\abdul\Dropbox (MIT)\PhD Thesis\Madina\Cities\Beirut\NEW_SCENARIOS_June2023\SCENARIO"
for file in os.listdir(folder+"0"):
    if '.geojson' in file:
        print (file)
        for scenario in [0, 2, 5, 6]:
            gdf = gpd.read_file(folder + str(scenario) + "\\" + file)
            print(f"{scenario = }\tcount = {gdf.shape[0]}")


# A Pedestrian Flow Simulation Workflow
## Data Inputs
## SImulation Parameters
## OD-Pairs and thier parameters
## Imterpriting Output